In [23]:
#importing required libraries
import pandas as pd
import numpy as np
import requests

In [76]:
#Creating a dataframe to store the information_table from the html page.
df_scraped = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

#Readjusting the first row to be the column header
df_scraped.columns = df_scraped.iloc[0]
df_scraped.drop(df_scraped.index[0], inplace=True)
df_scraped.reindex()

df_scraped.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [77]:
#Selecting Bouroughs without the 'Not assigned' ones
df_scraped = df_scraped[~(df_scraped['Borough']=='Not assigned')]
df_scraped.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [79]:
df_scraped['Neighbourhood'] = df_scraped[['Postcode','Borough','Neighbourhood']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
df_scraped = df_scraped[['Postcode','Borough','Neighbourhood']].drop_duplicates()
df_scraped = df_scraped.reset_index(drop=True)
df_scraped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [83]:
for i,nghhood in enumerate(df_scraped['Neighbourhood']):
    if (nghhood == 'Not assigned'):
        df_scraped['Neighbourhood'][i] = df_scraped['Borough'][i]
    

In [85]:
df_scraped.head(11)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [86]:
df_scraped.shape

(103, 3)

In [96]:
#We are reading the data form csv file to a new dataframe for storage 
import pandas as pd
import requests
import io

url = 'http://cocl.us/Geospatial_data'

r = requests.post(url)
if r.ok:
    data = r.content.decode('utf8')
    df_geocoder = pd.read_csv(io.StringIO(data))
df_geocoder.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [102]:
#renaming the columns for merging.
df_geocoder = df_geocoder.rename(columns={'Postal Code':'Postcode'})
df_geocoder.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [105]:
#Merging the two dataframes to get the final dataframe with lat,lon co-ordinates.
df_scraped = pd.merge(df_scraped,df_geocoder,on='Postcode')
df_scraped.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [106]:
df_scraped.shape

(103, 5)